In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
import matplotlib.style as style
import scipy.stats as sps

from solardatatools import DataHandler
from solardatatools.dataio import load_redshift_data
from solardatatools.algorithms import Dilation
from spcqe.quantiles import SmoothPeriodicQuantiles
from spcqe.quantiles import save, load

In [3]:
style.use('seaborn-v0_8-notebook')
dist = sps.norm()

# Load data

In [4]:
query = {
    'siteid': 'TAEHC1041562',
    'api_key': os.environ.get('REDSHIFT_API_KEY'),
    'sensor': 0
    }
df = load_redshift_data(**query)
dh = DataHandler(df, convert_to_ts=True)
dh.run_pipeline(start_day_ix=918)


            *********************************************
            * Solar Data Tools Data Onboarding Pipeline *
            *********************************************

            This pipeline runs a series of preprocessing, cleaning, and quality
            control tasks on stand-alone PV power or irradiance time series data.
            After the pipeline is run, the data may be plotted, filtered, or
            further analyzed.

            Authors: Bennet Meyers and Sara Miskovich, SLAC

            (Tip: if you have a mosek [https://www.mosek.com/] license and have it
            installed on your system, try setting solver='MOSEK' for a speedup)

            This material is based upon work supported by the U.S. Department
            of Energy's Office of Energy Efficiency and Renewable Energy (EERE)
            under the Solar Energy Technologies Office Award Number 38529.

            


task list: 100%|██████████████████████████████████| 7/7 [00:03<00:00,  1.91it/s]



total time: 3.67 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              0.58s
Cleaning                   0.03s
Filtering/Summarizing      3.07s
    Data quality           0.02s
    Clear day detect       0.04s
    Clipping detect        2.23s
    Capacity change detect 0.78s



In [5]:
nvals_dil = 101
dil = Dilation(dh)
sig = dil.signal_dil
mat = sig[1:].reshape((nvals_dil, dil.ndays), order='F')
mat[:,~dh.daily_flags.no_errors] = np.nan
sig[1:] = mat.ravel(order='F')

# Transform data

In [6]:
q1 = np.linspace(0, 1, 11)
q1[0] = 0.02
q1[-1] = 0.98

spq = SmoothPeriodicQuantiles(
        num_harmonics=[12, 3],
        periods=[nvals_dil, 365.24225*nvals_dil],
        standing_wave=[True, False],
        trend=False,
        quantiles=q1,
        weight=10,
        problem='sequential',
        solver='qss',
        extrapolate='solar',
    )
spq.fit(sig)

100%|███████████████████████████████████████████| 11/11 [02:12<00:00, 12.05s/it]


## Save, load and compare

In [7]:
save('saved_data/spq1.spq', spq)

In [8]:
spq2 = load('saved_data/spq1.spq')

Checking the type of the loaded object

In [9]:
spq2

SmoothPeriodicQuantiles(extrapolate={'lower': (-0.1, 'input'),
                                     'upper': (np.float64(4.264890793923841),
                                               'output')},
                        num_harmonics=[12, 3], periods=[101, 36889.46725],
                        quantiles=array([0.02, 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9 , 0.98]),
                        solver='qss', standing_wave=[True, False], weight=10)

Assessing the fit quantiles are the same

In [10]:
np.allclose(spq.fit_quantiles, spq2.fit_quantiles)

True

Note : The object above is quite large (25 MB) because of the basis. Not saving the basis would save memory space.